In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import random
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.colors import n_colors
from plotly.subplots import make_subplots
init_notebook_mode(connected=True)
import cufflinks as cf
cf.go_offline()
from wordcloud import WordCloud , ImageColorGenerator
from PIL import Image
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import matplotlib.pyplot as plt
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/indian-food-101/indian_food.csv')
df=df.replace(-1,np.nan)
df=df.replace('-1',np.nan)
df.head()


In [ ]:
df.columns

So we coloumns with Name of dish, ingredients, diet, prep time, cook time, flavor profile, course, origin state, region

In [ ]:
df.shape

so we have 255 rows and 9 coloumns.
Let us check if any of the entries are null.


In [ ]:
 pd.isnull(df).sum()

So we have a few null values in prep_time, cook_time, falvor_profile, state and region

In [ ]:
reg_df = df.region.value_counts().reset_index()
reg_df.columns = ['region','count']
reg_df = reg_df.sample(frac=1)
fig = px.bar(reg_df,x='region',y='count',title='Number of dishes based on regions',
             color_discrete_sequence=['#316394'])
fig.show()

# Let us visualize prominant cuisine and favor profile of each state  

In [ ]:
flavor_df = df.flavor_profile.value_counts().reset_index()
flavor_df.columns = ['flavor_profile','count']
flavor_df = flavor_df.sample(frac=1)
fig = px.bar(flavor_df,x='flavor_profile',y='count',title='Number of dishes based on flavor',
             color_discrete_sequence=['#316394'])
fig.show()


In [ ]:

pie_df = df.flavor_profile.value_counts().reset_index()
pie_df.columns = ['flavor','count']
fig = px.pie(pie_df, values='count', names='flavor', title='Proportion of Flavor Profiles',
             color_discrete_sequence=['#FF7F0E', '#00B5F7','#AB63FA','#00CC96'])
fig.show()

# We can deduce that indians love spicy and sweet flavor profile

Let us get state wise distribution of spicy and sweet flavor profile

In [ ]:
flavor_profile_list = ['sweet', 'spicy']

import geopandas as gpd


for flavor_profile in flavor_profile_list:
    shp_gdf = gpd.read_file('../input/india-gis-data/India States/Indian_states.shp')
    profile = df[df['flavor_profile']==flavor_profile]
    prof_df = profile.state.value_counts().reset_index()
    prof_df.columns = ['state','count']
    merged = shp_gdf.set_index('st_nm').join(prof_df.set_index('state'))
    fig, ax = plt.subplots(1, figsize=(10, 10))
    ax.axis('off')
    ax.set_title('State-wise Distribution of flavor:'+ str(flavor_profile),
                 fontdict={'fontsize': '15', 'fontweight' : '3'})
    fig = merged.plot(column='count', cmap='Wistia', linewidth=0.5, ax=ax, edgecolor='0.2',legend=True)

   

# Let us visualize word cloud of common ingredients in a spicy flavored dish  

In [ ]:
word_spicy_df  = df[(df['flavor_profile']=='spicy')].reset_index()

ingredients = []
for i in range(0,len(dessert_df)):
    text = word_spicy_df['ingredients'][i].split(',')
    text = ','.join(text)
    ingredients.append(text)
    text = ' '.join(ingredients)

wordcloud = WordCloud(width = 400, height = 400, background_color ='white', 
                min_font_size = 10).generate(text)                  
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis('off') 
plt.show()

# For Maharashtra state mini - info 

In [ ]:
mah_df = df[df['state']=='Maharashtra']

total_dishes = mah_df.shape[0]

course_df = mah_df['course'].value_counts().reset_index()
course_df.columns = ['course','num']

diet_df = mah_df['diet'].value_counts().reset_index()
diet_df.columns = ['diet','num']

fig = make_subplots(
    rows=2, cols=2,subplot_titles=('Total Dishes','Dishes by Courses','', ''),
    specs=[[{'type': 'indicator'},{'type': 'bar','rowspan': 2} ],
          [ {'type': 'pie'} , {'type': 'pie'}]])

fig.add_trace(go.Indicator(
    mode = 'number',
    value = int(total_dishes),
    number={'font':{'color': 'blue','size':50}},
),row=1, col=1)


fig.add_trace(go.Bar(
    x=course_df['course'],y=course_df['num'],
    marker={'color': 'orange'},  
    text=course_df['num'],
    name='dishes by courses',
    textposition ='auto'),row=1, col=2)

fig.add_trace(go.Pie(labels=diet_df['diet'], 
                     values=diet_df['num'],textinfo='percent',
                    marker= dict(colors=['green','red'])),row=2, col=1)


fig.update_layout(
    title_text='Maharashtra Food-Mini Infograph',template='plotly',
    title_x=0.5)

fig.show()

Top 10 snacks with shortest cooking time

In [ ]:
snack_df = df[df['course']=='snack']

short_sort_snack_df = snack_df.sort_values(['cook_time'],ascending=True).iloc[:10,:]

fig = px.bar(short_sort_snack_df,y='name',x='cook_time',
             orientation='h',color='cook_time',
            labels={'name':'Name of snack','cook_time':'Cooking time (minutes)'})
fig.show()

Snacks with largest cooking time


In [ ]:
long_sort_snack_df = snack_df.sort_values(['cook_time'],ascending=False).iloc[3:13,:]

fig = px.bar(long_sort_snack_df,y='name',x='cook_time',
             orientation='h',color='cook_time',
            labels={'name':'Name of snack','cook_time':'Cooking time (minutes)'})
fig.show()